In [1]:
import math
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

/Users/logan/Documents/projects/repos/function-approximation/venv/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:258: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

This model approximates the function f(x) = 1/1+e^-x

In [3]:
import struct

def int_to_64bit_list(number):
    binary_str = format(number, '064b')
    return [int(bit) for bit in binary_str]

def float_to_64bit_list(f):
    packed = struct.pack('!d', f)
    bits = ''.join(f'{byte:08b}' for byte in packed)
    return [int(bit) for bit in bits]

In [4]:
# test dataset
function = lambda x: math.sin(x)

ints = [x for x in range(10_000)]

input = [int_to_64bit_list(x) for x in ints]

output = [float_to_64bit_list(function(x)) for x in ints]



In [5]:
t = torch.Tensor(input)
t[500]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 0., 1., 0., 0.])

In [6]:
class Data(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.labels = outputs

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

In [7]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()  
        self.layer1 = nn.Linear(64, 256) # 64 bits in
        self.layer2 = nn.Linear(256, 256 * 5)  
        self.output = nn.Linear(256 * 5, 64) # 64 bits out

    def forward(self, x):
        # try relu to begin with
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.output(x)
        return x

In [8]:
training_dataset = Data(input, output)
training_loader = torch.utils.data.DataLoader(training_dataset,batch_size=256)

model = Model().to(device)

loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

for epoch in range(1000):
    for data in training_loader:
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        loss_size = loss(outputs, labels.to(device))
        loss_size.backward()
        optimizer.step()

    print(f"Loss: {loss_size.item()}")

torch.save(model.state_dict(), "model")

Loss: 148.9356689453125
Loss: 148.76669311523438
Loss: 148.60240173339844
Loss: 148.262939453125
Loss: 147.93243408203125
Loss: 147.65029907226562
Loss: 147.498779296875
Loss: 147.831298828125
Loss: 146.85520935058594
Loss: 146.63583374023438
Loss: 145.88592529296875
Loss: 145.81008911132812
Loss: 145.4483642578125
Loss: 145.20248413085938
Loss: 146.6939697265625
Loss: 145.1061248779297
Loss: 145.39105224609375
Loss: 143.91952514648438
Loss: 143.70062255859375
Loss: 143.79530334472656
Loss: 145.66278076171875
Loss: 146.73123168945312
Loss: 145.22755432128906
Loss: 143.8217315673828
Loss: 145.0723876953125
Loss: 146.39353942871094
Loss: 144.2838897705078
Loss: 142.37893676757812
Loss: 143.2325897216797
Loss: 142.34799194335938
Loss: 143.06533813476562
Loss: 143.3657684326172
Loss: 143.36524963378906
Loss: 141.36968994140625
Loss: 141.3603057861328
Loss: 143.22479248046875
Loss: 144.58197021484375
Loss: 141.54196166992188
Loss: 140.1295166015625
Loss: 141.23504638671875
Loss: 140.9772186

In [ ]:
with torch.no_grad():
  val = torch.tensor(input[500], dtype=torch.float32).to(device)
  answer = torch.tensor(output[500], dtype=torch.float32).to(device)

  outputs = model(val)  # Get predictions from the model.
  print(outputs.data)
  print(answer)
# print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

tensor([-6.7785e-01, -4.7360e+00,  5.8760e-01,  6.0990e-01,  7.5151e-01,
         6.5074e-01,  6.2571e-01,  5.9408e-01,  6.0190e-01,  4.5404e-01,
         1.0964e-01, -5.3351e-01,  9.9573e-02,  6.1304e-03,  2.3560e-02,
         4.0875e-02,  8.3050e-02,  2.6840e-01,  2.3909e-01, -9.4117e-02,
        -6.5678e-02,  1.3150e-01,  9.8387e-02,  9.9108e-02,  1.1590e-01,
         3.9549e-02,  1.0780e-01,  2.8574e-01, -1.4106e-01, -1.4607e-01,
         1.6567e-01, -2.0342e-01,  1.0628e-01,  1.2744e-02,  5.7347e-02,
        -2.0378e-01,  1.4350e-01,  2.0978e-01, -1.7601e-01,  9.9285e-02,
         1.2650e-01,  2.1908e-01, -1.2678e-01,  1.3858e-01, -1.1783e-01,
         2.1543e-01,  5.1218e-02,  4.2658e-02, -4.9893e-02, -1.9746e-01,
        -3.4787e-01, -1.4053e-01, -1.9032e-04, -7.1840e-02, -7.0026e-02,
        -6.0925e-03, -1.7909e-02, -1.3124e-01, -3.1149e-02,  7.5167e-02,
        -1.8330e-01,  4.2943e-02,  1.4329e-01,  1.7939e-02], device='mps:0')
tensor([1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,